## Advanced Data Analytics - Algorithms and Machine Learning
## 31005
### Harrison Cole
### 12962712

### Section 1 - Imports
Imports libraries and type-definitions for use throughout the program.

In [1185]:
import abc
import math

import numpy as np
import pandas as pd
import random
import traceback

from typing import Callable, Optional, Union, Tuple, List, Dict
from sklearn.datasets import load_iris, load_wine
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split

### Section 2 - Utility Function and Type Definitions
Defines utility functions and types for (re)use throughout the program.

In [1186]:
def require(value: Optional[any], field: str) -> any:
    """
    A mechanism for asserting the presence of a value, and raising an exception
    in the case of its absence.
    :param value:
    The value whose presence is being checked.
    :param field:
    A diagnostic tag indicating which value is absent.
    """
    if value is None:
        raise ValueError(f'Missing required value: "{field}".')
    return value


def default(value: Optional[any], otherwise: any) -> any:
    """
    A mechanism for checking for the presence of a value, and supplying a default value
    in the case of its absence.
    :param value:
    The value whose presence is being checked.
    :param otherwise:
    The default value to return in the case of it's absence.
    """
    return otherwise if value is None else value


def value_counts(elements, normalise: bool = True) -> tuple:
    """
    A mechanism for counting the occurrences of each unique value in a set of elements.
    :param elements:
    The set of elements.
    :param normalise:
    Whether or not to return the relative frequencies of the unique values.
    :return:
    The values and their corresponding representation within the set of elements
    as a tuple of arrays.
    """
    values, counts = np.unique(elements, return_counts=True)
    if normalise:
        return values, counts / np.sum(counts)
    return values, counts


def majority_class_index(elements):
    """
    A mechanism for returning the index of the class with the greatest representation
    in a set of elements.
    :param elements:
    The set of elements.
    """
    _, counts = value_counts(elements, normalise=False)
    return np.argmax(counts)


FeatureType = Union[str, int]
NumericType = Union[int, float]
PredicateType = Callable[[any], bool]

### Section 3 - Data-structures, Interfaces and Implementations
Defines the API and data-structures available for use throughout this program. Where applicable, effort is taken
to program by contract against the interface rather than the implementation.

#### Section 3.1 - Split Criterion Metrics

In [1187]:
class SplitCriterionMetric(metaclass=abc.ABCMeta):
    """
    An interface for computing the measure of quality produced by splitting the set of elements across
    the axis of a given variable at each step of computation during the tree building process.
    """

    @abc.abstractmethod
    def compute(self, frequencies) -> float:
        """
        Computes a measure of quality, usually the homogeneity ("sameness") of the target class, represented by the
        frequencies of each target class instance within a subset of the dataset.
        :param frequencies:
        The frequencies of each target class instance within this subset.
        :return:
        A floating point value where higher values indicate a higher degree of homogeneity.
        """
        pass


class Entropy(SplitCriterionMetric):

    def compute(self, frequencies) -> float:
        """
        Computes the entropy of the target class within a subset of the dataset.
        :param frequencies:
        The frequencies of each class instance within this subset.
        :return:
        A measure of the randomness of the distribution of each target class instance within this subset.
        """
        eps=1e-9
        return -(frequencies * np.log2(frequencies + eps)).sum()


class GiniImpurity(SplitCriterionMetric):

    def compute(self, frequencies) -> float:
        """
        Computes the Gini impurity of the target class within a subset of the dataset.
        :param frequencies:
        The frequencies of each class instance within this subset.
        :return:
        A measure of how often a randomly chosen element from the dataset would be incorrectly labelled if
        it was labelled according to the distribution of class instances within this subset.
        """
        return 1 - np.sum(np.square(frequencies))

#### Section 3.2 - Pivot

In [1188]:
class Pivot:
    """
    A component class that captures and describes an arbitrary predicate that is
    used as a pivot point for splitting a set of elements.
    i.e.
    elements = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    predicate = lambda e: e <= 5
    pivot = Pivot(predicate, (, ))
    splits = [pivot.split(e) for e in elements]
    [T, T, T, T, T, T, F, F, F, F, F]
    """

    def __init__(self, predicate: PredicateType, info: Tuple[any, any, str, str]):
        self.__predicate = require(predicate, 'predicate')
        self.__info = require(info, 'info')

    @property
    def predicate(self) -> PredicateType:
        """
        A property returning the predicate captured in this pivot.
        """
        return self.__predicate

    def attribute(self) -> any:
        """
        The value of the variable being pivoted upon.
        """
        return self.__info[0]

    def point(self) -> any:
        """
        The value(s) of the pivot point.
        """
        return self.__info[1]

    def true_condition(self) -> str:
        """
        The affirmative textual representation of the predicate.
        """
        return self.__info[2]

    def false_condition(self) -> str:
        """
        The negative textual representation of the predicate.
        :return:
        """
        return self.__info[3]

    def split(self, value: any) -> bool:
        """
        A mechanism for applying the predicate upon an element.
        :param value:
        The value upon which the predicate is applied.
        """
        return self.predicate(value)

    @staticmethod
    def continuous(attribute: FeatureType, point: NumericType) -> 'Pivot':
        """
        A static factory method for building a pivot that operates upon continuous (numerical)
        values.
        :param attribute:
        The name (str) or index (int) that represents the key of the attribute value
        upon which the pivot is applied within each element of a set of homogenous elements.
        :param point:
        The discrete value that represents the pivot point.
        :return:
        A pivot in the form of: lambda value: value[attribute] <= point
        """
        def predicate(value: any) -> bool:
            return value[attribute] <= point
        return Pivot(predicate=predicate, info=(attribute, point, '<=', '>'))

    def __str__(self, condition: bool = True) -> str:
        """
        The human-intelligible, textual representation of this pivot.
        :param condition:
        The negation of the predicate, if false.
        """
        operator: str = self.true_condition() if condition else self.false_condition()
        return f'x[{self.attribute()}] {operator.ljust(2)} {self.point()}'


class NumericalPivotCandidate:
    """
    A component data-structure for tracking the set of parameters that best splits a
    continuous attribute.
    """

    def __init__(self, feature: FeatureType, gain: float, probe: float):
        self.__feature = feature
        self.__gain = gain
        self.__probe = probe

    def feature(self) -> FeatureType:
        """
        The name (str) or index (int) that represents the key of the attribute that is being
        used as a feature.
        """
        return require(self.__feature, 'feature')

    def gain(self) -> float:
        """
        The gain yielded by this combination of feature and probe value.
        """
        return require(self.__gain, 'gain')

    def probe(self) -> float:
        """
        The best probe value tested thus far.
        """
        return require(self.__probe, 'probe')

    def update(self, feature: FeatureType, gain: float, probe: float) -> bool:
        """
        Compares a new parameter set with the previously best seen parameter set and
        updates the internal state of this data-structure if the new parameter set
        yields a better gain.
        :param feature:
        The feature being evaluated.
        :param gain:
        The gain produced by this split.
        :param probe:
        The probe value used in producing this split.
        :return:
        True if this feature combination yielded a better gain than that yielded
        by a previous combination, otherwise, False.
        """
        if gain < self.gain():
            return False
        self.__feature = feature
        self.__gain = gain
        self.__probe = probe
        return True

    @staticmethod
    def initial() -> 'NumericalPivotCandidate':
        """
        A static factory method for initialising a default NumericalPivotCandidate data-structure
        that has not seen previous parameter combinations.
        :return:
        """
        return NumericalPivotCandidate(feature=0, gain=-math.inf, probe=1.0)

    def __str__(self):
        """
        The human-intelligible, textual representation of the optimal parameter set.
        :return:
        """
        return f'feature: {self.__feature}, gain: {self.__gain}, probe: {self.__probe}'


#### Section 3.3 - Node

In [1189]:
class Node(metaclass=abc.ABCMeta):
    """
    An interface that encapsulates the logic of determining the class membership of an element
    within the hierarchy of a decision tree.
    """

    @abc.abstractmethod
    def eval(self, element: any) -> any:
        """
        Computes the class membership of an element, typically by means of traversing the node hierarchy
        recursively.
        :param element:
        The element whose class we wish to determine.
        :return:
        The class membership of the given element.
        """
        raise NotImplementedError('Node#eval')

    @abc.abstractmethod
    def depth(self) -> int:
        """
        Computes the number of levels (children) beneath this specific node, inclusive
        of the current node.
        """
        pass

    def prune(self) -> 'Node':
        """
        A mechanism for producing locally optimal or more efficient node configurations
        dependent upon a nodes internal state. This method provides no default implementation
        as optimisation is not necessary, although desirable, to the proper functioning of
        a decision tree.
        """
        return self

    @staticmethod
    def terminate(value: any) -> 'Node':
        """
        A static factory method for producing a terminal (leaf) node with the given value.
        :param value:
        The class membership of this terminal node.
        """
        return TerminalNode(value=value)

    @staticmethod
    def branch(pivot: 'Pivot', true_branch: Optional['Node'] = None, false_branch: Optional['Node'] = None) -> 'Node':
        """
        A static factory method for producing a branch (internal) node with the given pivot and true/false branch nodes.
        Used to handle mappings with a single split.
        :param pivot:
        The predicate upon which elements are pivoted.
        :param true_branch:
        The branch elements are directed to when the predicate evaluates true.
        :param false_branch:
        The branch elements are directed to when the predicate evaluates false.
        """
        return BranchNode(pivot=pivot, true_branch=true_branch, false_branch=false_branch)

    @staticmethod
    def lookup(mapping: Dict[any, 'Node'], feature: FeatureType) -> 'Node':
        """
        A static factory method for producing a branch (internal) node with the given attribute mappings.
        Used to handle mappings with an arbitrary number of splits.
        :param mapping:
        The dictionary of arbitrary key to node mappings.
        :param feature:
        The feature whose value is applied as the lookup key against the given mapping.
        """
        return LookupNode(mapping=mapping, feature=feature)


class TerminalNode(Node):
    """
    An implementation of the Node interface that statically resolves the class membership of an element.
    Analogous to a leaf node in standard decision tree implementations.
    """

    def __init__(self, value: any):
        self.__value = require(value, 'value')

    def eval(self, element: any) -> any:
        """
        Computes the class membership of the given element by statically mapping it to this node's value.
        """
        return self.value

    def depth(self) -> int:
        """
        This implementation has no children, therefore its depth is always 1.
        """
        return 1

    @property
    def value(self) -> any:
        """
        The class membership value that this node evaluates to.
        """
        return self.__value


class BranchNode(Node):
    """
    An implementation of the Node interface that dynamically resolves the class membership of an element.
    Analogous to an internal node in standard decision tree implementations.
    Used to handle mappings with a single split.
    """

    def __init__(self, pivot: 'Pivot', true_branch: 'Node', false_branch: 'Node'):
        self.__pivot = require(pivot, 'pivot')
        self.__nodes = np.asarray([false_branch, true_branch])

    def eval(self, element: any) -> any:
        """
        Dynamically computes the class membership of the given element by mapping it to another branch, by means of
        applying the pivot's predicate, and then recursing until arriving at a terminal node.
        """
        # Optimisation: removed the conditional branch to eliminate the cost incurred by miss-predicting a branch.
        # previous form: branch: Node = self.true_branch if self.pivot.split(element) else self.false_branch
        index: int = int(self.pivot.split(element))
        branch: Node = self.nodes[index]
        return branch.eval(element)

    def depth(self) -> int:
        """
        Computes the depth of this node and its deepest child.
        """
        levels = [child.depth() for child in self.nodes]
        return 1 + np.amax(levels, initial=0)

    def prune(self) -> 'Node':
        """
        This mechanism attempts to optimise the internal structure of this node.
        - eliminates the branch condition if possible.
        """
        if isinstance(self.true_branch, TerminalNode) and isinstance(self.false_branch, TerminalNode):
            if self.true_branch.value == self.false_branch.value:
                return self.true_branch
        return self

    @property
    def pivot(self) -> 'Pivot':
        """
        The pivot condition of this node.
        """
        return self.__pivot

    @property
    def nodes(self):
        """
        The array of nodes representing each branch.
        Always of length 2 in the form [false_branch, true_branch]
        """
        return self.__nodes

    @property
    def true_branch(self) -> 'Node':
        """
        The node applied when the pivot condition evaluates truthfully.
        """
        return self.nodes[int(True)]

    @property
    def false_branch(self) -> 'Node':
        """
        The node applied when the pivot condition evaluates falsely.
        """
        return self.nodes[int(False)]


class LookupNode(Node):
    """
    An implementation of the Node interface that dynamically resolves the class membership of an element.
    An extended version of the internal node in standard decision tree implementations that is used to handle
    mappings with an arbitrary number of splits.
    """

    def __init__(self, mapping: Dict[any, 'Node'], feature: FeatureType):
        self.__mapping = require(mapping, 'mapping')
        self.__feature = require(feature, 'feature')

    def eval(self, element: any) -> any:
        """
        Dynamically computes the class membership of the given element by mapping it to another branch, by means of
        looking up the feature value within the node's mapping, and then recursing until arriving at a terminal node.
        """
        key = element[self.feature]
        lookup: Node = self.mapping.get(key, None)
        if lookup is None:
            return 0
        return lookup.eval(element)
        # lookup: Node = require(self.mapping.get(key, None), f'lookup => {key}')
        # return lookup.eval(element)

    def depth(self, level: int = 0) -> int:
        """
        Computes the depth of this node and its deepest child.
        """
        levels = [child.depth() for child in self.mapping.values()]
        return 1 + np.amax(levels, initial=0)

    def prune(self) -> 'Node':
        """
        This mechanism attempts to optimise the internal structure of this node.
        - eliminates the mapping if there is only one element.
        N.B. This type of node could be subject to much further optimisation efforts.
        """
        size = len(self.mapping)
        if size == 1:
            return next(iter(self.mapping.values()))
        return self

    @property
    def mapping(self) -> Dict[any, 'Node']:
        """
        The internal mapping between feature values and nodes.
        """
        return self.__mapping

    @property
    def feature(self) -> FeatureType:
        """
        The feature whose values are mapped by this node.
        """
        return self.__feature


#### Section 3.4 - Decision Tree Builders

In [1190]:
# A mapping of all available implementations of the SplitCriterionMetric interface.
metrics: Dict[str, SplitCriterionMetric] = {
    'entropy': Entropy(),
    'gini': GiniImpurity()
}


# The default implementation of the SplitCriterionMetric to use in tree construction.
DEFAULT_CRITERION: str = 'entropy'


def get_metric(name: str) -> SplitCriterionMetric:
    """
    A utility function for safely retrieving a split criterion metric of the given name.
    """
    key: str = name if name in metrics.keys() else DEFAULT_CRITERION
    return require(metrics.get(key), f'metric => {name}')


class DecisionTreeBuilder(metaclass=abc.ABCMeta):
    """
    An interface that encapsulates the logic of constructing the internal node hierarchy of decision trees.
    """

    @abc.abstractmethod
    def build(self, x, y, **kwargs) -> 'Node':
        """
        The public method responsible for constructing the node hierarchy according to some node construction
        algorithm, i.e., CART, ID3, C4.5, etc.
        :param x:
        The set of data elements.
        :param y:
        The set of class membership labels corresponding to each element in the set of data elements.
        :param kwargs:
        Any additional parameters passed to the node construction algorithm.
        :return:
        The root node of a decision tree representing the node hierarchy.
        """
        raise NotImplementedError('DecisionTreeBuilder#build')

    def compute_metric(self, attributes, criterion: str) -> float:
        """
        A mechanism for computing the split value for the given collection of attributes.
        :param attributes:
        The set of attribute values.
        :param criterion:
        The name of the split criterion metric to use.
        :return:
        The floating point representation of the metric that was computed upon the relative frequencies
        of each attribute's value within the set of values.
        """
        _, frequencies = value_counts(attributes, normalise=True)
        metric: SplitCriterionMetric = get_metric(criterion)
        return metric.compute(frequencies=frequencies)

    # TODO: ensure correctness...
    # TODO: optimise
    def information_gain_categorical(self, target, feature, criterion: str) -> float:
        """
        A mechanism for computing the split value for the given collection of categorical attributes.
        :param target:
        The set of target values.
        :param feature:
        The set of feature values.
        :param criterion:
        The name of the split criterion metric to use.
        :return:
        The information gain yielded by performing the split against this feature.
        """
        total_info = self.compute_metric(target, criterion=criterion)
        split_info = self.compute_metric(feature, criterion=criterion)
        result: float = total_info - split_info
        # print(f'CATEGORICAL: total => {total_info}, feature: {split_info}, result: {result}, criterion: {criterion}')
        return result

    # TODO: ensure correctness...
    # TODO: refactor...
    # TODO: optimise
    def information_gain_continuous(self, target, feature, point, criterion: str) -> float:
        """
        A mechanism for computing the split value for the given collection of continuous attributes split at the
        given point value.
        :param target:
        The set of target values.
        :param feature:
        The set of feature values.
        :param point:
        The pivot point value.
        :param criterion:
        The name of the split criterion metric to use.
        :return:
        The information gain yielded by performing the split against this feature at the given pivot point.
        """
        size, split_indices = len(target), np.asarray([feature <= point, feature > point])
        total_info = self.compute_metric(target, criterion=criterion)
        split_info = np.sum(np.asarray([(self.compute_metric(feature[index], criterion=criterion) * (np.count_nonzero(index) / size)) for index in split_indices]))
        result: float = total_info - split_info
        # print(f'CONTINUOUS: total => {total_info}, feature: {split_info}, result: {result}, criterion: {criterion}')
        return result

    @staticmethod
    def factory(implementation: str, **kwargs) -> 'DecisionTreeBuilder':
        """
        A static factory method for constructing an instance of DecisionTreeBuilder.
        :param implementation:
        The name of the implementation to use, i.e., ID3.
        :param kwargs:
        A map of arguments to supply the constructor of the DecisionTreeBuilder implementation.
        :return:
        A constructed instance of DecisionTreeBuilder.
        """
        factories = {
            'ID3': ID3DecisionTreeBuilder
        }
        constructor = require(factories.get(implementation, None), implementation)
        return constructor(**kwargs)

    @staticmethod
    def default() -> 'DecisionTreeBuilder':
        """
        Constructs a reasonably selected default implementation of the DecisionTreeBuilder interface.
        Currently the default implementation is ID3.
        """
        return DecisionTreeBuilder.factory('ID3')


class ID3DecisionTreeBuilder(DecisionTreeBuilder):
    """
    An implementation of the DecisionTreeBuilder interface that constructs internal node hierarchy of a decision tree
    as per the specification of the Iterative Dichotomiser 3 (ID3) algorithm by Ross Quinlan.

    The specification can be found at the following location: https://en.wikipedia.org/wiki/ID3_algorithm#Algorithm
    """
    
    def build(self, x, y, criterion: str = DEFAULT_CRITERION) -> 'Node':
        """
        The public API for constructing the ID3 node hierarchy for the given set of elements and class membership labels.
        :param x:
        The set of data elements.
        :param y:
        The set of class membership labels.
        :param criterion:
        The name of the split criterion metric to use.
        :return:
        The constructed node hierarchy.
        """
        data = x.copy()
        data[y.name] = y
        return self._build(data=data, features=x.columns, target=y.name, criterion=criterion)

    def _build(self, data, features, target: FeatureType, criterion: str, parent_class=None) -> 'Node':
        """
        The internal API for constructing the ID3 node hierarchy.
        :param data:
        The set of elements and target values.
        :param features:
        The set of features.
        :param target:
        The target attribute to predict.
        :param criterion:
        The name of the split criterion metric to use.
        :param parent_class:
        The majority class of the previous node (if any).
        :return:
        The constructed ID3 node hierarchy.
        """
        target_subset = data[target]
        classes = np.unique(target_subset)
        choices = len(classes)

        # base case #1
        # There are no examples in the subset, which happens when no example in the parent set was found to match a
        # specific value of the selected attribute.
        if len(data) <= 0:
            return Node.terminate(parent_class)

        # base case #2
        # Every element of the subset belongs to the same class.
        if choices <= 1:
            return Node.terminate(classes[0])

        majority_class = classes[majority_class_index(elements=target_subset)]

        # base case #3
        # There are no more attributes to be selected, but the examples still do not belong to the same class.
        if len(features) <= 0:
            return Node.terminate(majority_class)

        gains = np.asarray([self.information_gain_categorical(target=target_subset, feature=data[feature], criterion=criterion) for feature in features])
        best_feature = features[np.argmax(gains)]
        attribute = data[best_feature]
        remaining_features = [feature for feature in features if feature != best_feature]

        subtree: Node

        # noinspection PyBroadException
        try:
            subtree = self._build_continuous(data=data, features=remaining_features, target=target, criterion=criterion, parent_class=majority_class, best_feature=best_feature, attribute=attribute)
        except Exception as e:
            # traceback.print_exception(e)
            subtree = self._build_categorical(data=data, features=remaining_features, target=target, criterion=criterion, parent_class=majority_class, best_feature=best_feature, attribute=attribute)

        while (pruned := subtree.prune()) != subtree:
            subtree = pruned

        return subtree

    def _build_continuous(self, data, features, target, criterion: str, parent_class, best_feature, attribute) -> 'Node':
        probes = self.create_probe_values(attribute.min(), attribute.max())
        candidate: NumericalPivotCandidate = NumericalPivotCandidate.initial()
        for probe in probes:
            gain = self.information_gain_continuous(target=data[target], feature=attribute, point=probe, criterion=criterion)
            candidate.update(feature=best_feature, gain=gain, probe=probe)

        def build_subtree(indices) -> 'Node':
            return self._build(data=data[indices], features=features, target=target, criterion=criterion, parent_class=parent_class)

        pivot: Pivot = Pivot.continuous(candidate.feature(), candidate.probe())
        true_branch = build_subtree(data[candidate.feature()] <= candidate.probe())
        false_branch = build_subtree(data[candidate.feature()] > candidate.probe())

        return Node.branch(pivot=pivot, true_branch=true_branch, false_branch=false_branch)

    def _build_categorical(self, data, features, target, criterion: str, parent_class, best_feature, attribute) -> 'Node':
        values = np.unique(attribute)
        mapping: Dict[any, Node] = {}

        for value in values:
            subset = data.where(attribute == value).dropna()
            subtree = self._build(subset, features=features, target=target, criterion=criterion, parent_class=parent_class)
            mapping[value] = subtree

        return Node.lookup(mapping=mapping, feature=best_feature)

    def create_probe_values(self, minima, maxima):
        weights = np.arange(0.0, 1.0, 0.05)
        return np.asarray([point * minima + (1.0 - point) * maxima for point in weights])


In [1191]:
class Model(metaclass=abc.ABCMeta):

    def compile(self, **kwargs):
        pass

    @abc.abstractmethod
    def fit(self, x, y, **kwargs):
        raise NotImplementedError('Model#fit')

    @abc.abstractmethod
    def predict(self, x, **kwargs):
        raise NotImplementedError('Model#predict')


class DecisionTree(Model):

    # TODO: default value (most common class..?)
    __builder: 'DecisionTreeBuilder' = DecisionTreeBuilder.default()
    __root: Optional['Node'] = None
    __depth: Optional[int] = None

    def compile(self, **kwargs):
        previous: DecisionTreeBuilder = self.__builder
        try:
            builder = DecisionTreeBuilder.factory(kwargs['implementation'], **kwargs)
        except (KeyError, ValueError):
            builder = previous
        self.__builder = builder

    def fit(self, x, y, **kwargs):
        self.__root = self.builder.build(x, y, **kwargs)
        self.__depth = None

    def predict(self, x, **kwargs):
        tree: Node = self.root
        samples = x.to_dict(orient='records')
        return np.asarray([tree.eval(sample) for sample in samples])

    @property
    def builder(self) -> 'DecisionTreeBuilder':
        return require(self.__builder, 'builder')

    @property
    def root(self) -> 'Node':
        return require(self.__root, 'root')

    def depth(self) -> int:
        depth: int
        if self.__root is None:
            # the tree has not been built.
            depth = 0
        elif self.__depth is not None:
            # the depth is cached as it has previously been calculated and the tree has not changed.
            depth = self.__depth
        else:
            # the depth has not been calculated prior to this invocation.
            depth = self.__depth = self.root.depth()
        return depth

### Section 4 - Implementation

In [1192]:
def debug_tree(node, depth: int = 0):
    if isinstance(node, DecisionTreeClassifier):
        print(export_text(node))
        return
    if isinstance(node, DecisionTree):
        debug_tree(node.root)
        return

    padding = ('|   ' * depth)[:-3] + '---'

    def p(o):
        print(f'{str(depth).ljust(2)} {padding} {o}')

    if isinstance(node, TerminalNode):
        p(f'class: {node.value}')
    elif isinstance(node, BranchNode):
        p(f'{node.pivot.__str__(condition=True)}')
        debug_tree(node.true_branch, depth=depth+1,)
        p(f'{node.pivot.__str__(condition=False)}')
        debug_tree(node.false_branch, depth=depth+1)
    elif isinstance(node, LookupNode):
        for (k, v) in node.mapping.items():
            p(f'lookup: {node.feature} {k}')
            debug_tree(v, depth=depth+1)
    else:
        raise ValueError(f'Unexpected node: {node}')


def junk_mixed_dataset(samples: int = 100):
    attribute_pool = {
        'wind_direction': ['N', 'S', 'E', 'W'],
        'tide': ['Low', 'High'],
        'swell_forecasting': ['small', 'medium', 'large'],
        'good_waves': ['Yes', 'No'],
        'temp': '',
        'hello': ''
    }
    df = pd.DataFrame(columns=attribute_pool.keys())
    np.random.seed(42)
    for i in range(samples):
        df.loc[i, 'wind_direction'] = str(np.random.choice(attribute_pool['wind_direction'], 1)[0]) # categorical
        df.loc[i, 'tide'] = str(np.random.choice(attribute_pool['tide'], 1)[0]) # categorical
        df.loc[i, 'swell_forecasting'] = str(np.random.choice(attribute_pool['swell_forecasting'], 1)[0]) # categorical
        df.loc[i, 'good_waves'] = str(np.random.choice(attribute_pool['good_waves'], 1)[0]) # categorical
        df.loc[i, 'good_waves_int'] = int(np.random.choice([0, 1], 1)[0]) # numeric somewhat correlated
        df.loc[i, 'temp'] = int(np.random.random() * 26) + 1 # numeric but irrelevant
        df.loc[i, 'hello'] = 'world' # categorical but irrelevant

    target_attr = df['good_waves'].copy()
    df.drop(labels=['good_waves'], axis=1, inplace=True)
    return df, target_attr

datasets = {
    'iris': lambda: load_iris(return_X_y=True, as_frame=True),
    'wine': lambda: load_wine(return_X_y=True, as_frame=True),
    'junk': lambda: junk_mixed_dataset(150)
}

for (name, provider) in datasets.items():
    [elements, labels] = provider()
    random_state: int = 42
    train_split_size: float = 0.7
    debug: bool = True

    for metric in ['entropy', 'gini']:

        if debug:
            print(f'--- DATASET -- {name} -- {metric} ---')
            print(f'Elements:\n\n{elements[:5]}\n')
            print(f'Targets:\n\n{labels[:5]}\n')

        train_x, test_x = train_test_split(elements, train_size=train_split_size, random_state=random_state)
        train_y, test_y = train_test_split(labels, train_size=train_split_size, random_state=random_state)

        ours: DecisionTree = DecisionTree()
        theirs: DecisionTreeClassifier = DecisionTreeClassifier(criterion=metric)

        def evaluate(text: str, classifier):
            try:
                if isinstance(classifier, DecisionTree):
                    classifier.fit(train_x, train_y, criterion=metric)
                else:
                    classifier.fit(train_x, train_y)

                if debug:
                    print()
                    print()
                    print(f'--- {text} ---')
                    debug_tree(classifier)
                    print(f'--- {text} ---')
                    print()
                    print()

                predictions = classifier.predict(test_x)
                accuracy = accuracy_score(test_y, predictions)
                return accuracy, predictions
            except Exception as e:
                return f'Exception occurred during processing: {e}', None

        us = evaluate('ours', ours)
        them = evaluate('theirs', theirs)

        if them[1] is not None:
            differences = np.nonzero(us[1] - them[1])
        else:
            differences = None

        print(f'--- RESULTS -- {name} -- {metric} ---')
        print(f'Our Accuracy:   {us[0]}')
        print(f'Their Accuracy: {them[0]}')
        print(f'Differences: {differences if them[1] is not None else "N/A"}')
        print(f'Ours:        {us[1][differences] if them[1] is not None else "N/A"}')
        print(f'Theirs:      {them[1][differences] if them[1] is not None else "N/A"}')
        print(f'Correct:     {np.asarray(test_y)[differences] if them[1] is not None else "N/A"}')
        print()


# # TODO: python 3.6 compatibility (for google colab)
# # TODO: CRISP-DM

--- DATASET -- iris -- entropy ---
Elements:

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2

Targets:

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64



--- ours ---
0  --- x[sepal width (cm)] <= 2.99
1  |--- x[petal width (cm)] <= 1.3
2  |   |--- class: 1
1  |--- x[petal width (cm)] >  1.3
2  |   |--- x[petal length (cm)] <= 5.02
3  |   |   |--- x[sepal length (cm)] <= 6.0200000000000005
4  |   |   |   |--- class: 2
3  |   |   |--- x[sepal length (cm)] >  6.0200000000000005
4  |   |   |   |--- class: 1
2  |   |--- x[petal length (cm)] >  5.02
3  |   |   |--- class: 2
